# 01. Data Processing & Advanced Feature Engineering

**Objective:** Transform raw SQLite data into a high-quality dataset for predicting football match outcomes.

**Key Steps:**
1.  **Data Loading & Cleaning:** Handle missing values and types.
2.  **Core Feature Engineering:** Betting odds, Rolling Form.
3.  **Advanced Feature Engineering:**
    *   **Elo Ratings:** Dynamic strength assessment.
    *   **Head-to-Head (H2H):** Historical performance against specific opponents.
    *   **Top Scorer Indicator:** identifying star players impact.
    *   **Playing Style Clustering:** K-Means on Team Attributes.
4.  **Output:** Save to `data/afterprocessing/processed_matches.csv`.

In [26]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from datetime import datetime

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading & Cleaning

In [ ]:
DB_PATH = "../data/database.sqlite"

conn = sqlite3.connect(DB_PATH)
df_matches = pd.read_sql("SELECT * FROM Match", conn)
df_teams = pd.read_sql("SELECT * FROM Team", conn)
df_player_atts = pd.read_sql("SELECT * FROM Player_Attributes", conn)
df_team_atts = pd.read_sql("SELECT * FROM Team_Attributes", conn)
conn.close()

print(f"Matches: {df_matches.shape}, Teams: {df_teams.shape}")

In [ ]:
# Sort by Date
df_matches['date'] = pd.to_datetime(df_matches['date'])
df_matches = df_matches.sort_values('date').reset_index(drop=True)

# Clean Goals
df_matches = df_matches.dropna(subset=['home_team_goal', 'away_team_goal'])

# Target: 2=Home Win, 1=Draw, 0=Away Win
conditions = [
    df_matches['home_team_goal'] > df_matches['away_team_goal'],
    df_matches['home_team_goal'] == df_matches['away_team_goal'],
    df_matches['home_team_goal'] < df_matches['away_team_goal']
]
df_matches['target'] = np.select(conditions, [2, 1, 0], default=-1)

## 2. Advanced Feature Engineering

### 2.1 Elo Rating & Head-to-Head (H2H)
We calculate Elo dynamic ratings AND track history between specific pairs of teams (H2H).

In [ ]:
def calculate_elo_prob(rating1, rating2):
    return 1.0 / (1 + 10 ** ((rating2 - rating1) / 400.0))

def update_elo(rating_home, rating_away, goal_diff, result, k=20, home_field_adv=100):
    rating_home_adj = rating_home + home_field_adv
    prob_home = calculate_elo_prob(rating_home_adj, rating_away)
    
    if goal_diff <= 1: G = 1
    elif goal_diff == 2: G = 1.5
    else: G = (11 + goal_diff) / 8.0
        
    new_rating_home = rating_home + k * G * (result - prob_home)
    new_rating_away = rating_away + k * G * ((1 - result) - (1 - prob_home))
    return new_rating_home, new_rating_away

# Init State
team_elos = {team_id: 1500 for team_id in df_teams['team_api_id'].unique()}
h2h_data = {} # Key: tuple(sorted(id1, id2)), Value: {id1_wins: 0, id2_wins: 0, draws: 0}

k_factor = 20

# Feature Columns to build
home_elo_col, away_elo_col, elo_probs_home = [], [], []
h2h_home_wins_last3, h2h_away_wins_last3, h2h_draws_last3 = [], [], []

for idx, row in df_matches.iterrows():
    h_id, a_id = row['home_team_api_id'], row['away_team_api_id']
    
    # --- 1. ELO ---
    h_elo = team_elos.get(h_id, 1500)
    a_elo = team_elos.get(a_id, 1500)
    
    home_elo_col.append(h_elo)
    away_elo_col.append(a_elo)
    elo_probs_home.append(calculate_elo_prob(h_elo + 100, a_elo))
    
    # --- 2. H2H (Before updating with current result) ---
    # Get history
    pair_key = tuple(sorted((h_id, a_id)))
    history = h2h_data.get(pair_key, [])
    # History is list of results from perspective of team with smaller ID: 
    # e.g. [(match1_winner_id), (match2_winner_id), ...]
    
    # We want last 3 matches only
    recent_history = history[-3:]
    
    w_home, w_away, w_draw = 0, 0, 0
    for winner in recent_history:
        if winner == h_id: w_home += 1
        elif winner == a_id: w_away += 1
        else: w_draw += 1
            
    h2h_home_wins_last3.append(w_home)
    h2h_away_wins_last3.append(w_away)
    h2h_draws_last3.append(w_draw)
    
    # --- UPDATE POST MATCH ---
    h_goals, a_goals = row['home_team_goal'], row['away_team_goal']
    goal_diff = abs(h_goals - a_goals)
    
    # Result for Elo
    if h_goals > a_goals: res_elo = 1.0; res_h2h = h_id
    elif h_goals == a_goals: res_elo = 0.5; res_h2h = 'draw'
    else: res_elo = 0.0; res_h2h = a_id
        
    # Update Elo
    new_h, new_a = update_elo(h_elo, a_elo, goal_diff, res_elo, k=k_factor)
    team_elos[h_id] = new_h
    team_elos[a_id] = new_a
    
    # Update H2H
    if pair_key not in h2h_data: h2h_data[pair_key] = []
    h2h_data[pair_key].append(res_h2h)

df_matches['home_elo'] = home_elo_col
df_matches['away_elo'] = away_elo_col
df_matches['elo_diff'] = df_matches['home_elo'] - df_matches['away_elo']
df_matches['elo_prob_home'] = elo_probs_home

df_matches['h2h_home_wins'] = h2h_home_wins_last3
df_matches['h2h_away_wins'] = h2h_away_wins_last3
df_matches['h2h_draws'] = h2h_draws_last3

### 2.2 Top Scorer Indicator

In [ ]:
df_player_atts['date'] = pd.to_datetime(df_player_atts['date'])
df_player_atts['year'] = df_player_atts['date'].dt.year
# Approx Top Player > 82 overall
top_players_per_year = df_player_atts[df_player_atts['overall_rating'] >= 82].groupby('year')['player_api_id'].unique().to_dict()

def check_top_player(row, side):
    year = row['date'].year
    # Check year and year-1
    top_list = set(top_players_per_year.get(year, [])) | set(top_players_per_year.get(year-1, []))
    count = 0
    for i in range(1, 12):
        pid = row[f'{side}_player_{i}']
        if pd.notna(pid) and pid in top_list:
            count += 1
    return count

df_matches['home_star_players'] = df_matches.apply(lambda x: check_top_player(x, 'home'), axis=1)
df_matches['away_star_players'] = df_matches.apply(lambda x: check_top_player(x, 'away'), axis=1)

### 2.3 Playing Style Clustering

In [ ]:
# Preprocess Team Attributes (Forward Fill to prevent data loss)
df_team_atts['date'] = pd.to_datetime(df_team_atts['date'])
style_cols = ['buildUpPlaySpeed', 'buildUpPlayPassing', 'chanceCreationPassing', 
              'chanceCreationCrossing', 'chanceCreationShooting', 
              'defencePressure', 'defenceAggression', 'defenceTeamWidth']

# Sort and Forward Fill per team
df_team_atts.sort_values(['team_api_id', 'date'], inplace=True)
df_team_atts[style_cols] = df_team_atts.groupby('team_api_id')[style_cols].ffill()

df_team_atts_clean = df_team_atts.dropna(subset=style_cols).copy()

if not df_team_atts_clean.empty:
    scaler = StandardScaler()
    X_style = scaler.fit_transform(df_team_atts_clean[style_cols])
    kmeans = KMeans(n_clusters=3, random_state=42)
    df_team_atts_clean['style_cluster'] = kmeans.fit_predict(X_style)
    
    df_team_atts_clean['date'] = pd.to_datetime(df_team_atts_clean['date'])
    df_team_atts_clean.sort_values('date', inplace=True)
    df_matches = df_matches.sort_values('date')
    
    for side in ['home', 'away']:
        atts = df_team_atts_clean[['team_api_id', 'date', 'style_cluster']].copy()
        atts.columns = [f'{side}_team_api_id', 'date', f'{side}_style']
        df_matches = pd.merge_asof(df_matches, atts, on='date', by=f'{side}_team_api_id', direction='backward')
    
    df_matches[['home_style', 'away_style']] = df_matches[['home_style', 'away_style']].fillna(-1)
else:
    df_matches['home_style'] = -1
    df_matches['away_style'] = -1

### 2.4 Form, Odds & Feature Divergence
Calculating Betting Odds Implied Probabilities and Rolling Form (Points, GP, GF, GA).

In [ ]:
# --- 4. ROLLING FORM & STATS (Detailed) ---
# Create a long-format database of team performances to calculate rolling stats correctly
def build_team_stats(matches):
    # Home stats
    home = matches[['date', 'match_api_id', 'home_team_api_id', 'home_team_goal', 'away_team_goal', 'target']].copy()
    home.columns = ['date', 'match_api_id', 'team_id', 'goals_for', 'goals_against', 'target']
    home['is_home'] = 1
    home['points'] = np.where(home['target'] == 2, 3, np.where(home['target'] == 1, 1, 0))
    home['win'] = (home['target'] == 2).astype(int)
    home['draw'] = (home['target'] == 1).astype(int)
    
    # Away stats
    away = matches[['date', 'match_api_id', 'away_team_api_id', 'away_team_goal', 'home_team_goal', 'target']].copy()
    away.columns = ['date', 'match_api_id', 'team_id', 'goals_for', 'goals_against', 'target']
    away['is_home'] = 0
    away['points'] = np.where(away['target'] == 0, 3, np.where(away['target'] == 1, 1, 0))
    away['win'] = (away['target'] == 0).astype(int)
    away['draw'] = (away['target'] == 1).astype(int)
    
    # Combine and sort
    team_stats = pd.concat([home, away], ignore_index=True).sort_values(['team_id', 'date'])
    return team_stats

team_stats = build_team_stats(df_matches)

# Calculate Rolling Stats
windows = [5, 10]
cols_to_roll = ['points', 'goals_for', 'goals_against', 'win', 'draw']

# Group by team
g = team_stats.groupby('team_id')

for w in windows:
    for col in cols_to_roll:
        # Shift 1 to exclude current match from average
        team_stats[f'last{w}_{col}'] = g[col].transform(lambda x: x.shift(1).rolling(w, min_periods=1).mean())

# Fill NaNs (for first matches)
roll_cols = [c for c in team_stats.columns if c.startswith('last')]
team_stats[roll_cols] = team_stats[roll_cols].fillna(team_stats[roll_cols].mean()) # Simple mean fill for early games

# Merge back to matches
# We need to map team_stats back to home and away teams in df_matches
home_stats = team_stats[team_stats['is_home'] == 1][['match_api_id'] + roll_cols].copy()
home_stats.columns = ['match_api_id'] + [f'home_{c}' for c in roll_cols]

away_stats = team_stats[team_stats['is_home'] == 0][['match_api_id'] + roll_cols].copy()
away_stats.columns = ['match_api_id'] + [f'away_{c}' for c in roll_cols]

df_matches = df_matches.merge(home_stats, on='match_api_id', how='left')
df_matches = df_matches.merge(away_stats, on='match_api_id', how='left')

# Calculate Differences (Home - Away)
for col in roll_cols:
    df_matches[f'diff_{col}'] = df_matches[f'home_{col}'] - df_matches[f'away_{col}']

# Odds Implied Probabilities
if 'B365H' in df_matches.columns:
    df_matches['B365_prob_H'] = 1 / df_matches['B365H']
    df_matches['B365_prob_D'] = 1 / df_matches['B365D']
    df_matches['B365_prob_A'] = 1 / df_matches['B365A']
    
    # Divergence: Elo Probability vs Market Probability
    df_matches['prob_diff_home'] = df_matches['elo_prob_home'] - df_matches['B365_prob_H']

## 3. Save

In [ ]:
import os
os.makedirs('../data/afterprocessing', exist_ok=True)

final_cols = [
    'id', 'match_api_id', 'date', 'season', 'league_id', 
    'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'target',
    'home_elo', 'away_elo', 'elo_diff', 'elo_prob_home',
    'h2h_home_wins', 'h2h_away_wins', 'h2h_draws',
    'home_star_players', 'away_star_players',
    'home_style', 'away_style',
    'B365H', 'B365D', 'B365A', 'B365_prob_H', 'B365_prob_D', 'B365_prob_A',
    'prob_diff_home'
]
# Add dynamic rolling columns
roll_feats = [c for c in df_matches.columns if c.startswith('home_last') or c.startswith('away_last') or c.startswith('diff_last')]
final_cols.extend(roll_feats)

cols_to_save = [c for c in final_cols if c in df_matches.columns]

df_matches[cols_to_save].to_csv('../data/afterprocessing/processed_matches.csv', index=False)
print("Data saved to data/afterprocessing/processed_matches.csv")